Build and train a graph convolutional neural network using PyTorch Geometric for the node property prediction task.

We will use ogbn-products dataset.

## OGBN-Products

The ogbn-products dataset is an undirected and unweighted graph, representing an Amazon product co-purchasing network. Nodes represent products sold in Amazon, and edges between two products indicate that the products are purchased together. Node features are generated by extracting bag-of-words features from the product descriptions followed by a Principal Component Analysis to reduce the dimension to 100.

The task is to predict the category of a product in a multi-class classification setup, where the 47 top-level categories are used for target labels.

In [ ]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.0.1+cu118


Download the necessary packages for PyG. Make sure that your version of torch matches the output from the cell above. In case of any issues, more information can be found on the [PyG's installation page](https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html).

In [ ]:
# Install torch geometric
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-{torch.__version__}.html
!pip install torch-geometric
!pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html


In [ ]:
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
import numpy as np
from torch_geometric.typing import SparseTensor

## Load and Preprocess the Dataset

In [ ]:
dataset_name = 'ogbn-products'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
data = dataset[0]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

# data = data.to(device)
# split_idx = dataset.get_idx_split()
# train_idx = split_idx['train'].to(device)

Device: cuda


In [ ]:
data

Data(num_nodes=2449029, x=[2449029, 100], y=[2449029, 1], adj_t=[2449029, 2449029, nnz=123718280])

This dataset is very big and if you try to run it as it is on colab, you may get an out of memory error.

One solution is to use batching and train on subgraphs. Here, we will just make a smaller dataset so that we can train it in one go.

In [ ]:
# We need to have edge indxes to make a subgraph. We can get those from the adjacency matrix.
data.edge_index = torch.stack([data.adj_t.__dict__["storage"]._row, data.adj_t.__dict__["storage"]._col])

# We will only use the first 100000 nodes.
sub_nodes = 100000
sub_graph = data.subgraph(torch.arange(sub_nodes))

# Update the adjaceny matrix according to the new graph
sub_graph.adj_t = SparseTensor(
    row=sub_graph.edge_index[0],
    col=sub_graph.edge_index[1],
    sparse_sizes=None,
    is_sorted=True,
    trust_data=True,
)

sub_graph = sub_graph.to(device)

sub_graph


Data(num_nodes=100000, x=[100000, 100], y=[100000, 1], adj_t=[100000, 100000, nnz=2818046], edge_index=[2, 2818046])

In [ ]:
# Spilt data into train validation and test set
split_sizes = [int(sub_nodes*0.8),int(sub_nodes*0.05),int(sub_nodes*0.15)]
indices = torch.arange(sub_nodes)
np.random.shuffle(indices.numpy())
split_idx = {s:t for t,s in zip(torch.split(indices, split_sizes, dim=0), ["train", "valid", "test"])}
split_idx

{'train': tensor([ 4147, 51486, 39502,  ..., 51924, 37323, 53748]),
 'valid': tensor([92772, 71789, 44747,  ..., 18380,  5719, 25899]),
 'test': tensor([95861, 56140, 47449,  ..., 79326, 38807, 49482])}

In [ ]:
print(f"Feature Length of each node: {data.x.shape[1]}")

Feature Length of each node: 100


## GCN Model

Now we will implement our GCN model!